In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
def cleansing(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail주소제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]' # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text    


https://search.naver.com/search.naver?date_from=20200101&date_option=8&date_to=20200201&dup_remove=1&nso=p%3Afrom20200101to20200201&post_blogurl=&post_blogurl_without=&query=치즈볼&sm=tab_pge&srchby=all&st=sim&where=post&start=1
https://search.naver.com/search.naver?date_from=20200101&date_option=8&date_to=20200201&dup_remove=1&nso=p%3Afrom20200101to20200201&post_blogurl=&post_blogurl_without=&query=치즈볼&sm=tab_pge&srchby=all&st=sim&where=post&start=11
https://search.naver.com/search.naver?date_from=20200202&date_option=8&date_to=20200301&dup_remove=1&nso=p%3Afrom20200202to20200301&post_blogurl=&post_blogurl_without=&query=치즈볼&sm=tab_pge&srchby=all&st=sim&where=post&start=1
https://search.naver.com/search.naver?date_from=20200202&date_option=8&date_to=20200301&dup_remove=1&nso=p%3Afrom20200202to20200301&post_blogurl=&post_blogurl_without=&query=치즈볼&sm=tab_pge&srchby=all&st=sim&where=post&start=11
https://search.naver.com/search.naver?date_from=20200302&date_option=8&date_to=20200401&dup_re

In [14]:
#네이버 블로그 매달 100페이지씩
keyword  = '치즈볼'
month_start = ['20200101','20200202','20200302','20200402','20200502','20200602','20200702','20200802','20200902','20201002']
month_end = ['20200201','20200301','20200401','20200501','20200601','20200701','20200801','20200901','20201001','20201014']
title_list=[]
date_list=[]
for a in range(len(month_start)):

    for b in range(0,100):
        try:
            url = 'https://search.naver.com/search.naver?date_from={0}&date_option=8&date_to={1}&dup_remove=1&nso=p%3Afrom{0}to{1}&post_blogurl=&post_blogurl_without=&query={2}&sm=tab_pge&srchby=all&st=sim&where=post&start={3}'.format(month_start[a],month_end[a],keyword,b*10 +1)
            
            r = requests.get(url)
            html = r.content
            soup = BeautifulSoup(html, "html.parser")
            dates = soup.find_all("dd", {'class':"txt_inline"}) #날짜
            title_tags = soup.find_all("a", {'class':"sh_blog_title _sp_each_url _sp_each_title"})  #한페이지당 모든제목 가져옴
            content_tags = soup.find_all("dd", {'class':"sh_blog_passage"})     #한페이지당 모든 짧은 설명 가져옴


            for k in range(len(title_tags)):
                date = dates[k].text #k번째 블로그날짜
                title_tag = title_tags[k]       #k번째 블로그제목
                title_tag = title_tag["title"]  #k번쨰 블로그제목 중 text만
                content_tag = content_tags[k].text  #k번째 블로그 짧은글 중 text만
                
                one_sentence = title_tag + ' ' + content_tag    #제목과 글을 더해서 한문장으로
                one_sentence = cleansing(one_sentence)
                
                date_list.append(date)
                title_list.append(one_sentence)


        except:
            print(month_start[a],b, '에러')

df = pd.DataFrame({'date':date_list, 'title':title_list})
df.to_csv('../prj_sns_trend_private/crawling_naver_blog_치즈볼.csv')



In [8]:
df

,date,title
0,2020.10.08.,비비큐 네고왕 광희나는 메이플버터갈릭치킨 황금올리브 치즈볼 닭껍질튀김 JMT 솔직후...
1,2020.10.12.,에어프라이어 냉동치킨 애슐리 치킨 스윗 콤보치즈볼 후기 모짜렐라치즈볼도 금방 만들어...
2,7일 전,후기 밥솥 시금치적채애호박 소고기 이유식 아기치즈 치즈볼 흑 아기간식치즈볼 렌자렌...
3,2020.10.02.,이마트 트레이더스 순살치킨 크림 치즈볼 구입 솔직후기 크림치즈볼입니다 겉면은 바삭...
4,2020.10.08.,치킨콤보 애슐리 치즈볼과 간장치킨 강추 홈스토랑 오픈함 치즈가 장난아니죠 깜놀함 ...
...,...,...
995,2020.10.02.,좋은 치즈볼은 태어나서 가장 맛있게 먹어본 그맛이다 단백하고 부드러운 치즈맛에 반...
996,2020.10.12.,역삼 No Brand Burger 노브랜드버거 샌드위치와 샐러드 까지 들어오면 볼...
997,7일 전,치즈볼은 아주 쫄깃한 느낌 겉이 블랙이라 안엔 어떨까했는데 치즈가 듬뿍 들었고 약...
998,7일 전,미니소프트 치즈 케이크 7790원 순면 베게커버 2p 14490원 스피어민트 ...


In [15]:
#카페 매달 100페이지 크롤링

keyword  = [ '치즈볼']
month_start = ['20200101','20200202','20200302','20200402','20200502','20200602','20200702','20200802','20200902','20201002']
month_end = ['20200201','20200301','20200401','20200501','20200601','20200701','20200801','20200901','20201001','20201014']
title_list=[]
date_list=[]
for a in range(len(month_start)):

    for b in range(0,100):
        try:
            url='https://search.naver.com/search.naver?where=article&ie=utf8&query={0}&prdtype=0&\
            t=0&st=rel&date_option=6&date_from={1}&date_to={2}&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&\
            board=&sm=tab_pge&nso=so:r,p:from{1}to{2},a:all&start={3}'.format(keyword,month_start[a],month_end[a],b*10+1)
            
            r = requests.get(url)
            html = r.content
            soup = BeautifulSoup(html, "html.parser")
            dates = soup.find_all("dd", {'class':"txt_inline"}) #날짜
            title_tags = soup.find_all("a", {'class':"sh_cafe_title"})  #한페이지당 모든제목 가져옴
            content_tags = soup.find_all("dd", {'class':"sh_cafe_passage"})     #한페이지당 모든 짧은 설명 가져옴


            for k in range(len(title_tags)):
                date = dates[k].text #k번째 블로그날짜
                title_tag = title_tags[k]       #k번째 블로그제목
                title_tag = title_tag["title"]  #k번쨰 블로그제목 중 text만
                content_tag = content_tags[k].text  #k번째 블로그 짧은글 중 text만
                
                one_sentence = title_tag + ' ' + content_tag    #제목과 글을 더해서 한문장으로
                one_sentence = cleansing(one_sentence)
                
                date_list.append(date)
                title_list.append(one_sentence)

        except:
            print(month_start[a],b, '에러')


df = pd.DataFrame({'date':date_list, 'title':title_list})
df.to_csv('../prj_sns_trend_private/crawling_naver_cafe_치즈볼.csv')



In [31]:
df1 = pd.read_csv('../prj_sns_trend_private/crawling_naver_cafe_20200101.csv')

In [32]:
for a in range(len(month_start) -1):
    b = a+1
    df2 = pd.read_csv('../prj_sns_trend_private/crawling_naver_cafe_{}.csv'.format(month_start[b]))
    df1 = pd.concat([df1,df2], ignore_index=True)
    
df1

,Unnamed: 0,date,title
0,0,2020.01.04.,지난 달 아들 둘과 저녁 외식 후 달달한 도넛을 간식으로 일심 동체 하여 구매 하...
1,1,2020.01.14.,치킨보다 치즈볼이 더 맛있네요 적당히 달고 맛있어서 순삭이네요 이럴 줄 알았으면 ...
2,2,2020.01.05.,시중에 파는 냉동 치즈볼이나 치킨집 치즈볼은 양도 적고 너무 비싸서 만들어 보기로...
3,3,2020.01.03.,아들이 치즈볼을 너무 좋아해서 치즈볼 만들어봤어요 생각보다 괜찮은거 같아요 치즈를...
4,4,2020.01.14.,페리카나에서 치킨 시켰는데 치즈볼 서비스로 주셨는데 넘 맛있네요 아기도 잘먹고 치...
...,...,...,...
9995,995,5일 전,외식업계도 부캐 열풍사이드 메뉴 인기몰이 bhc치킨은 대표 사이드 메뉴는 2018년...
9996,996,2일 전,250300 칼로리 인거 알고있었냐 나 그거 안뒤로 충격먹어서 치즈볼 못먹겠음
9997,997,어제,안녕하세요 우리들의 알뜰한 쇼핑을 위해 치즈볼 제품 중 가장 인기있는 상품을 소개...
9998,998,2일 전,뿌링클 순살에 치즈볼 맛있겠다 77억


In [33]:
df1.drop([df1.columns[0]],axis=1, inplace=True)
df1.to_csv('../prj_sns_trend_private/crawling_naver_cafe.csv')